In [ ]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [ ]:
#I don't think this is still needed
for obj in decode_stacked(fc):
    ind = 0
    try: versions = list(obj['versions'].keys())
    except: continue
    try: release_dates = obj['time']
    except: continue
    info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
    for key in versions:
        try:
            info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
            if pkg in info[key]['dependencies']: ind = 1
        except: continue
    if ind :
        dependent[pkg].append({obj['name']: info})

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [ ]:
#user-info
#    pji
#        version 1.0
#        date
#        
#        version 1.1
#        date
#    gifnoc
#        version
#        date
        

In [ ]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append(obj['name'])

In [ ]:
for k in dependent.keys():
    for i in dependent[k]:
        print(i)
        #print(k, dependent[k][:20])
        #print("len = ", len(dependent[k]))
        #break

In [ ]:
#ajv-keywords
    #jsconscript-proxy
    #senasaikou-yeoman
    #webpack-egoist
#user-info
    #ajv-keywords
        

In [ ]:
for k in dependent.keys():
    print(dependent[k][:20])
    print("len = ", len(dependent[k]))
    break

In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=7)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for pkg in pkgs:
    dl = []
    for r in dt_range:
        url = base_url+r[0]+':'+r[1]+'/'+pkg
        try: r = requests.get(url)
        except: print('Timeout '+ pkg+'\n')
        
        try:
            result = r.content
            try:
                rj = json.loads(result.decode('utf-8', errors='ignore'))
                rj.pop('package', None)
                dl = dl + [rj]
            except: print('DecodeError '+ pkg+'\n')
        except: print('BadURL '+ pkg+'\n')
    #ent = {pkg: dl}    
    ent = {'name':pkg, 'dl': dl}

    coll.insert_one(ent)
    print(ent)

In [63]:
import math
total_list = []
start_date = []
end_date = []
download_list = []

for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    for k in ent:
        download_list.append(k['downloads'])
        start_date.append(k["start"])
#map, reduce, filter, lambda
    log_list = [math.log(x+1) for x in download_list]
    difference_list = []
    for i in range(1,len(log_list)-1):
        dif = log_list[i] - log_list[i-1]
        difference_list.append( (log_list[i] - log_list[i-1]))
        if dif > 2:
            print("Start date of spike: " +start_date[i])
        if dif < -1.5:
            print("Start date of fall: " +start_date[i])
    #print(difference_list)
#print(log_list)
#print(download_list)


Start date of spike: 2016-06-07
Start date of fall: 2016-06-23
Start date of fall: 2016-07-09
Start date of spike: 2016-08-10
Start date of fall: 2016-09-11
Start date of spike: 2016-09-19
Start date of fall: 2016-11-14
Start date of fall: 2016-12-16
Start date of fall: 2017-03-30
Start date of fall: 2017-04-23
Start date of spike: 2017-05-01
Start date of fall: 2017-06-10
Start date of fall: 2017-09-22
Start date of spike: 2017-09-30
Start date of fall: 2017-11-25
Start date of spike: 2017-12-19
Start date of fall: 2018-03-25
Start date of fall: 2018-04-10
Start date of spike: 2018-04-18
Start date of fall: 2018-05-28
Start date of spike: 2018-06-13
Start date of fall: 2018-06-21
Start date of fall: 2018-07-23
Start date of fall: 2018-08-24
Start date of spike: 2016-10-21
Start date of fall: 2016-11-06
Start date of fall: 2017-01-17
Start date of spike: 2017-01-25
Start date of fall: 2017-02-18
Start date of spike: 2017-03-14
Start date of fall: 2017-04-23
Start date of fall: 2017-05-

In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=7)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for k in dependent.keys():
    for i in dependent[k]:
        dl = []
        for r in dt_range:
            url = base_url+r[0]+':'+r[1]+'/'+i
            try: r = requests.get(url)
            except: print('Timeout '+ i+'\n')
        
            try:
                result = r.content
                try:
                    rj = json.loads(result.decode('utf-8', errors='ignore'))
                    rj.pop('package', None)
                    dl = dl + [rj]
                except: print('DecodeError '+ i+'\n')
            except: print('BadURL '+ i+'\n')
        #ent = {pkg: dl}    
        ent = {'name':i, 'dl': dl}
        coll.insert_one(ent)
        print(ent)

In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for r in coll.find():
    temp = r['name']
    print(temp)

In [56]:
import math, time, requests, sys, pymongo, json, datetime

dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]
#scaling up -- for pkg in pkgs?
print("For package " + pkg + ":")
for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    flag = 0
    for k in ent:
        if k["downloads"] > 10000: #ADDITION--------------
            flag = 1
    if flag == 1:    
        for k in ent:
            download_list.append(k["downloads"])
            start_date.append(k["start"])
#map, reduce, filter, lambda
            log_list = [math.log(x+1) for x in download_list]
   # difference_list = []
    if flag == 1:
            for i in range(1,len(log_list)-1):
                dif = log_list[i] - log_list[i-1]
       # difference_list.append(dif)
                if dif > 3.5:
                    print("Start date of spike for " + r['name'] +": " +start_date[i])
                if dif < -3.5:
                    print("Start date of fall for " + r['name'] +": " +start_date[i])


For package ajv-keywords:
Start date of spike for @webpack-contrib/schema-utils: 2018-04-02
Start date of spike for schema-utils: 2017-03-06
Start date of spike for schema-utils: 2017-05-01
Start date of spike for read-config-file: 2017-07-12
Start date of spike for yoshi: 2018-04-18
